<a href="https://colab.research.google.com/github/mavenceslau/Challenge-Data-Science-Alura-Cash/blob/master/notebooks/Notebook_03_Modelos_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semana 02 - Aprendendo com os dados: criando um modelo de previsão de inadimplência - Modelos de Machine Learning

Chegou a hora de criar os modelos de machine learning para classificação, além de usar métricas para testar a eficácia dos mesmos.

In [ ]:
import sys
sys.path.append('../')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from utils import config_exibicao, config_graficos, box, remove_outlier

Com a função **config_exibicao** iremos configurar a exibição de dataframes pandas, de modo que os mesmos possam exibir o maior número de informações nas células sem cortar conteudo.

E com a função **config_graficos** vamos configurar a exibição dos gráficos.

Mais informações sobre essas funções podem ser vistas no arquivo [utils.py](https://github.com/mavenceslau/Challenge-Data-Science-Alura-Cash/blob/master/notebooks/utils.py)

In [ ]:
config_exibicao()
config_graficos()

Importando a base de dados e exibindo os primeiros registros

In [ ]:
url = '../data/processed/alura_cash_tratado.csv'
dados = pd.read_csv(url)
dados.head()

,pessoa_idade,pessoa_salario_anual,pessoa_status_propriedade,pessoa_tempo_trabalho,emprestimo_motivo,emprestimo_pontuacao,emprestimo_valor_total,emprestimo_taxa_juros,emprestimo_inadimplente,emprestimo_renda_percentual,historico_inadimplencia,historico_tempo_credito
0,22,50000.0,Hipotecada,0,Pagamento de débitos,A,12800.0,5.79,0,0.26,N,2.0
1,26,84996.0,Alugada,0,Médico,B,9000.0,11.26,0,0.11,N,2.0
2,24,55782.0,Alugada,3,Melhora do lar,A,17000.0,7.51,0,0.30,N,2.0
3,25,125000.0,Própria,9,Pessoal,B,25000.0,10.59,0,0.20,N,2.0
4,23,44340.0,Própria,0,Empreendimento,C,3000.0,12.98,0,0.07,S,3.0


In [ ]:
print(f'O dataset possui {dados.shape[0]} linhas e {dados.shape[1]} colunas')

O dataset possui 32316 linhas e 12 colunas


## Aplicando Encoding nas váriaveis

In [ ]:
# Feita a alteração, onde o 'N' foi substituido por '0', e 'S' por '1'

for cliente in 'historico_inadimplencia':
    dados['historico_inadimplencia'] = dados['historico_inadimplencia'].replace({'N': 0, 'S': 1})

dados.head(10)

,pessoa_idade,pessoa_salario_anual,pessoa_status_propriedade,pessoa_tempo_trabalho,emprestimo_motivo,emprestimo_pontuacao,emprestimo_valor_total,emprestimo_taxa_juros,emprestimo_inadimplente,emprestimo_renda_percentual,historico_inadimplencia,historico_tempo_credito
0,22,50000.0,Hipotecada,0,Pagamento de débitos,A,12800.0,5.79000,0,0.26,0,2.0
1,26,84996.0,Alugada,0,Médico,B,9000.0,11.26000,0,0.11,0,2.0
2,24,55782.0,Alugada,3,Melhora do lar,A,17000.0,7.51000,0,0.30,0,2.0
3,25,125000.0,Própria,9,Pessoal,B,25000.0,10.59000,0,0.20,0,2.0
4,23,44340.0,Própria,0,Empreendimento,C,3000.0,12.98000,0,0.07,1,3.0
5,29,50000.0,Hipotecada,4,Médico,B,21850.0,12.42000,0,0.44,0,10.0
6,27,35000.0,Alugada,8,Pessoal,B,12000.0,9.91000,1,0.34,0,8.0
7,32,19200.0,Alugada,0,Empreendimento,B,5000.0,9.99000,1,0.26,0,6.0
8,22,70800.0,Alugada,6,Educativo,C,10000.0,12.61000,0,0.14,1,2.0
9,22,45000.0,Alugada,1,Empreendimento,E,10000.0,11.01363,1,0.22,0,2.0


In [ ]:
categorias = ['pessoa_status_propriedade', 'emprestimo_motivo', 'emprestimo_pontuacao']
dados2 = dados.drop(categorias, axis=1)
dados2.columns

Index(['pessoa_idade', 'pessoa_salario_anual', 'pessoa_tempo_trabalho',
       'emprestimo_valor_total', 'emprestimo_taxa_juros',
       'emprestimo_inadimplente', 'emprestimo_renda_percentual',
       'historico_inadimplencia', 'historico_tempo_credito'],
      dtype='object')

In [ ]:
ohe = OneHotEncoder(dtype=int)

colunas_ohe = ohe.fit_transform(dados[categorias]).toarray()
dados_final = pd.concat([dados2, pd.DataFrame(colunas_ohe, columns=ohe.get_feature_names_out(categorias))], axis=1)
dados_final.head()

,pessoa_idade,pessoa_salario_anual,pessoa_tempo_trabalho,emprestimo_valor_total,emprestimo_taxa_juros,emprestimo_inadimplente,emprestimo_renda_percentual,historico_inadimplencia,historico_tempo_credito,pessoa_status_propriedade_Alugada,pessoa_status_propriedade_Hipotecada,pessoa_status_propriedade_Outros,pessoa_status_propriedade_Própria,emprestimo_motivo_Educativo,emprestimo_motivo_Empreendimento,emprestimo_motivo_Melhora do lar,emprestimo_motivo_Médico,emprestimo_motivo_Pagamento de débitos,emprestimo_motivo_Pessoal,emprestimo_pontuacao_A,emprestimo_pontuacao_B,emprestimo_pontuacao_C,emprestimo_pontuacao_D,emprestimo_pontuacao_E,emprestimo_pontuacao_F,emprestimo_pontuacao_G
0,22,50000.0,0,12800.0,5.79,0,0.26,0,2.0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
1,26,84996.0,0,9000.0,11.26,0,0.11,0,2.0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
2,24,55782.0,3,17000.0,7.51,0,0.30,0,2.0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
3,25,125000.0,9,25000.0,10.59,0,0.20,0,2.0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0
4,23,44340.0,0,3000.0,12.98,0,0.07,1,3.0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0


## Verificando se a variável target está balanceada

In [ ]:
dados_final.emprestimo_inadimplente.value_counts()

0    25047
1     7269
Name: emprestimo_inadimplente, dtype: int64

In [ ]:
print(f'Existem {dados_final.emprestimo_inadimplente.value_counts()[0]} clientes em dia e ' +
      f'{dados_final.emprestimo_inadimplente.value_counts()[1]} clientes inadimplentes. O dataset não está balanceado.')

Existem 25047 clientes em dia e 7269 clientes inadimplentes. O dataset não está balanceado.


Irei utilizar a técnica de balanceamento de dados **oversampling**, que é basicamente duplicar, fazer a cópia dos nossos dados da classe minoritária até chegar no valor da classe majoritária. Então, ela fará cópia dos dados de clientes inadimplentes, ou seja, criaria novos dados até chegarmos ao número de dados dos clientes em dia. Para que não ocorra muitos dados idênticos, será utilizada a técnica SMOTE, que cria dados sintéticos próximos dos reais.

In [ ]:
#! pip install -U imbalanced-learn

In [ ]:
from imblearn.over_sampling import SMOTE

SEED = 25
smote = SMOTE(random_state = SEED)

In [ ]:
x = dados_final.drop('emprestimo_inadimplente', axis = 1)
y = dados_final['emprestimo_inadimplente']

In [ ]:
x_resampled, y_resampled = smote.fit_resample(x, y)

In [ ]:
dados_final = pd.concat([y_resampled, x_resampled], axis = 1)
dados_final

,emprestimo_inadimplente,pessoa_idade,pessoa_salario_anual,pessoa_tempo_trabalho,emprestimo_valor_total,emprestimo_taxa_juros,emprestimo_renda_percentual,historico_inadimplencia,historico_tempo_credito,pessoa_status_propriedade_Alugada,pessoa_status_propriedade_Hipotecada,pessoa_status_propriedade_Outros,pessoa_status_propriedade_Própria,emprestimo_motivo_Educativo,emprestimo_motivo_Empreendimento,emprestimo_motivo_Melhora do lar,emprestimo_motivo_Médico,emprestimo_motivo_Pagamento de débitos,emprestimo_motivo_Pessoal,emprestimo_pontuacao_A,emprestimo_pontuacao_B,emprestimo_pontuacao_C,emprestimo_pontuacao_D,emprestimo_pontuacao_E,emprestimo_pontuacao_F,emprestimo_pontuacao_G
0,0,22,50000.000000,0,12800.000000,5.790000,0.260000,0,2.000000,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
1,0,26,84996.000000,0,9000.000000,11.260000,0.110000,0,2.000000,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
2,0,24,55782.000000,3,17000.000000,7.510000,0.300000,0,2.000000,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
3,0,25,125000.000000,9,25000.000000,10.590000,0.200000,0,2.000000,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0
4,0,23,44340.000000,0,3000.000000,12.980000,0.070000,1,3.000000,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50089,1,26,16108.869916,2,5000.000000,12.031696,0.310000,0,3.371003,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
50090,1,25,39000.000000,4,19981.760589,14.441603,0.510000,0,2.875492,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
50091,1,30,66739.856964,6,13128.019714,15.484263,0.200235,0,4.536918,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
50092,1,22,18713.497684,1,4068.445427,15.103237,0.213422,0,4.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [ ]:
print(f'Existem {dados_final.emprestimo_inadimplente.value_counts()[0]} clientes em dia e ' +
      f'{dados_final.emprestimo_inadimplente.value_counts()[1]} clientes inadimplentes. O dataset está balanceado.')

Existem 25047 clientes em dia e 25047 clientes inadimplentes. O dataset está balanceado.
